In [30]:
import numpy as np
import pandas as pd
import re
import datetime
import warnings
import sys

from miscfns.misc_fns import db_con,describe_dupe_cor_ans, get_item_cor_ans, recode_as_omitted, timing_exclusion, remove_repeat_questions, combine_CIinfo, removed_record_count, clean_item_data
from miscfns.misc_fns import make_user_level_matrices, make_item_level_info, make_activity_level_info, make_user_level_info

from miscfns.misc_fns import color, teleg_msg

In [180]:
template = 16971
analysis_name = 'GBR_'+str(template)
results_path = 'C:\\Users\\VImmadisetty\\Downloads\\DATA\\DRCR\\BAR\\Results\\'+str(template)+'\\'

In [181]:
path = 'C:\\Users\\VImmadisetty\\Downloads\\DATA\\DRCR\\BAR\\Data\\'

In [182]:
activity_df = pd.read_csv(path + 'activity_info_'+str(template)+'.csv')
response_df = pd.read_csv(path + 'response_info_'+str(template)+'.csv')
content_df = pd.read_csv(path + 'content_info_all.csv')

In [183]:
print('Shape of activity_df :', activity_df.shape)
print('Total Students : ', activity_df['student_id'].nunique())
print('Total Activities : ', activity_df['activity_id'].nunique())
print('Total Enrollments : ', activity_df['kbs_enrollment_id'].nunique())
print('Template Id : ', activity_df['template_id'].unique())

print('-'*50)

print('Shape of response_df :', response_df.shape)
print('Total Students : ', response_df['student_id'].nunique())
print('Total Activities : ', response_df['activity_id'].nunique())
print('Total Content Items : ', response_df['content_item_id'].nunique())

print('-'*50)

print('Shape of content_df :', content_df.shape)
print('Total Content Items : ', content_df['content_item_id'].nunique())
print('Is Correct_answer column available? ', 'correct_answer' in content_df.columns)
#print('Data available in correct answer columns :', content_df['correct_answer'].nunique())

Shape of activity_df : (1071, 23)
Total Students :  1071
Total Activities :  1071
Total Enrollments :  1068
Template Id :  [16971]
--------------------------------------------------
Shape of response_df : (107100, 20)
Total Students :  1071
Total Activities :  1071
Total Content Items :  100
--------------------------------------------------
Shape of content_df : (3799, 11)
Total Content Items :  548
Is Correct_answer column available?  False


# Data handling

In [184]:
response_df.rename(columns={'history_db_id': 'historiesDb',
                            'student_id' : 'jasperUserId',
                            'activity_id' : 'sequenceId',
                            'item_position': 'position',
                            'section_title' : 'sectionTitle',
                            'item_section_position' : 'displaySeq',
                            'content_item_id' : 'contentItemId',
                            'content_item_name' : 'contentItemName',
                            'interaction_type': 'interactionType',
                            'milliseconds_used' : 'mSecUsed',
                            'is_scored' : 'scored',
                            'scored_response' : 'score',
                            'raw_response' : 'response',
                            'field_test' : 'fieldTest',
                           'item_status': 'responseStatus'}, inplace = True)
#response_df['correctAnswer'] = np.nan
response_df['sectionName'] = response_df['sectionTitle']

#removing responses with null display sequence
#response_df = response_df[~response_df['displaySeq'].isnull()]


In [185]:
#encoding responses to numbers
ans_dt = {'A': '1', 'B':'2', 'C':'3', 'D':'4'}
response_df['min_raw_answer'] = response_df['min_raw_answer'].replace(ans_dt)
response_df['max_raw_answer'] = response_df['max_raw_answer'].replace(ans_dt)

#concatenating raw responses to get them in single column
response_df.loc[response_df['min_raw_answer']==response_df['max_raw_answer'], 'response'] = response_df['min_raw_answer']
response_df.loc[response_df['min_raw_answer']!=response_df['max_raw_answer'], 'response'] = response_df['min_raw_answer'] + ',' + response_df['max_raw_answer']

In [186]:
#renaming the column names of original data to avoid changing the code
activity_df.rename(columns={'kbs_enrollment_id':'enrollment_id',
                           'date_created':'timestamp_created',
                           'date_completed':'timestamp_completed'}, inplace= True)

In [187]:
activity_df.rename(columns={'history_db_id': 'historiesDb',
                            'student_id' : 'jasperUserId',
                            'enrollment_id' : 'kbsEnrollmentId',
                            'activity_id' : 'sequenceId',
                            'template_id' : 'templateId',
                            'template_name' : 'sequenceName',
                            'sequence_title' : 'sequenceTitle',
                            'timestamp_created' : 'dateCreated',
                            'timestamp_completed' : 'dateCompleted',
                           'tutor_mode' : 'tutorMode',
                           'status':'sequenceStatus'}, inplace = True)

activity_df['dateCreated'] = pd.to_datetime(activity_df['dateCreated'], errors = 'coerce')
activity_df['dateCompleted'] = pd.to_datetime(activity_df['dateCompleted'], errors = 'coerce')


#merging responses and activity data and changing the names of statuses
print('Response_df size before merging with activity_df :', response_df.shape)
initi_cols = response_df.columns
response_df = pd.merge(response_df.drop(columns=['source_system']), activity_df.drop(columns=['source_system']), on = ['jasperUserId', 'sequenceId', 'historiesDb'], how = 'inner')

print('Response_df size before merging with activity_df:', response_df.shape)
print('New columns added from activity_df with activity_df:\n', set(response_df)^set(initi_cols))

Response_df size before merging with activity_df : (107100, 22)
Response_df size before merging with activity_df: (107100, 40)
New columns added from activity_df with activity_df:
 {'course_access_id', 'activity_type', 'sequenceStatus', 'total_scored_items_omitted', 'program', 'total_scored_items_answered_correct', 'total_scored_items_time_elapsed', 'tutorMode', 'dateCompleted', 'kbsEnrollmentId', 'activity_name', 'templateId', 'total_scored_items_answered', 'total_items_time_elapsed', 'total_scored_items_not_reached', 'dateCreated', 'total_items', 'total_scored_items', 'sequenceName', 'source_system'}


In [188]:
#changing kbs_enrollment_id to string as in cleaning function to remove 0 kbseids we are comparing as string
response_df['kbsEnrollmentId'].fillna(value = 0, inplace = True)
response_df['kbsEnrollmentId'] = response_df['kbsEnrollmentId'].astype('str')

* Not merging with content_df to see the implications

In [189]:
#renaming the column names of original data to avoid changing the code
content_df.rename(columns={'answer_index':'correct_answer'}, inplace= True)

In [190]:
content_df.rename(columns = {'content_item_id' : 'contentItemId',
                            'content_item_name' : 'contentItemName',
                            'interaction_type_name' : 'interactiontypename',
                            'count_choices' : 'countchoices',
                            'parent_item_id' : 'parentid',
                            'parent_item_name' : 'parentname',
                            'correct_answer':'correctAnswer'}, inplace = True)

content_df['correctAnswer'] = content_df['correctAnswer'].replace(ans_dt)

content_df = content_df[content_df['template_id']==template][['source_system', 'history_db_id', 'contentItemId', 'contentItemName',
                                                'interactiontypename', 'countchoices', 'correctAnswer']].drop_duplicates(ignore_index=True)

#category name vary based on problem, so check the data
#mergning with response_df
print('shape of response_df before merging with content_df :', response_df.shape)

response_df = pd.merge(response_df, content_df,
                       how = 'left',
                      on = ['contentItemId', 'contentItemName'])

print('shape of response_df after merging with content_df :', response_df.shape)

shape of response_df before merging with content_df : (107100, 40)
shape of response_df after merging with content_df : (107100, 45)


In [191]:
#section name col is empty, so using it for noting exam names
response_df.loc[response_df['templateId']==44, 'sectionName'] = 'Final exam'
response_df.loc[response_df['templateId']==46, 'sectionName'] = 'Midterm exam'
response_df.loc[response_df['templateId']==5755, 'sectionName'] = 'Diagnostic exam'
response_df.loc[response_df['templateId']==16968, 'sectionName'] = 'Mid-Term Practice Exam 1 - Summer20'
response_df.loc[response_df['templateId']==16969, 'sectionName'] = 'Mid-Term Practice Exam 2 - Summer20'
response_df.loc[response_df['templateId']==16970, 'sectionName'] = 'Simulated Practice Exam 1 - Summer20'
response_df.loc[response_df['templateId']==16971, 'sectionName'] = 'Simulated Practice Exam 2 - Summer20'

In [192]:
#responses summary
response_summary = response_df[response_df['sequenceStatus'].str.lower() == 'complete'].groupby(['templateId', 'sequenceName', 'sectionName', 'jasperUserId'])[['sequenceId']].agg('count').reset_index()
response_summary.rename(columns = {'sequenceId' : 'num_responses'}, inplace = True)

response_summary = response_summary.groupby(['templateId', 'sequenceName'])[['num_responses']].agg(['max', 'min', 'median', 'count'])
response_summary.columns = ['max_resp', 'min_resp', 'median_resp', 'num_users']

response_summary

,,max_resp,min_resp,median_resp,num_users
templateId,sequenceName,,,,
16971,Simulated MCQ Practice Exam 2 - Summer20,100,100,100.0,1071


In [193]:
test_map = pd.DataFrame()
test_map['templateId'] = [44, 46, 5755, 16968, 16969, 16970, 16971]
test_map['jasperSequenceName'] = ['Simulated MCQ Practice Exam',
                                 'Mid-Term MCQ Practice Exam',
                                 'MCQ Diagnostic Practice Exam',
                                 'Mid-Term MCQ Practice Exam 1 - Summer20',
                                 'Mid-Term MCQ Practice Exam 2 - Summer20',
                                 'Simulated MCQ Practice Exam 1 - Summer20',
                                 'Simulated MCQ Practice Exam 2 - Summer20']

test_map['sectionName'] = ['Final exam', 'Midterm exam', 'Diagnostic exam',
                          'Mid-Term Practice Exam 1 - Summer20', 'Mid-Term Practice Exam 2 - Summer20',
                          'Simulated Practice Exam 1 - Summer20',
                          'Simulated Practice Exam 2 - Summer20']

test_map['numQues'] = [200, 200, 150, 100, 100, 100, 100]
test_map['responseThreshold'] = 0.75
test_map['minutesAllowed'] = [360, 360, 270, 180, 180, 180, 180]

test_map

,templateId,jasperSequenceName,sectionName,numQues,responseThreshold,minutesAllowed
0,44,Simulated MCQ Practice Exam,Final exam,200,0.75,360
1,46,Mid-Term MCQ Practice Exam,Midterm exam,200,0.75,360
2,5755,MCQ Diagnostic Practice Exam,Diagnostic exam,150,0.75,270
3,16968,Mid-Term MCQ Practice Exam 1 - Summer20,Mid-Term Practice Exam 1 - Summer20,100,0.75,180
4,16969,Mid-Term MCQ Practice Exam 2 - Summer20,Mid-Term Practice Exam 2 - Summer20,100,0.75,180
5,16970,Simulated MCQ Practice Exam 1 - Summer20,Simulated Practice Exam 1 - Summer20,100,0.75,180
6,16971,Simulated MCQ Practice Exam 2 - Summer20,Simulated Practice Exam 2 - Summer20,100,0.75,180


# Connecting to Db for Repeats enrolls

In [194]:
try:
    engine
except:
    engine = db_con()

query_frt = """select
    distinct ph.id kbsenrollmentid
from
    kbs_billing.purchase_history ph
    join bi_reporting.vw_product_detail prd on ph.product_id = prd.product_id
where
    lower(prd.product_subtype) in ('free trial');
    """

try:
    frt_enrols
except:
    frt_enrols = pd.read_sql(sql = query_frt, con = engine)

query_olc = """select
    distinct ph.id kbsenrollmentid
from
    kbs_billing.purchase_history ph
    join bi_reporting.vw_product_detail prd on ph.product_id = prd.product_id
where
    lower(prd.product_subtype) in ('online companion')
    ;"""

try:
    olc_enrols
except:
    olc_enrols = pd.read_sql(sql = query_olc, con = engine)

query_hsg_repeat = """select
    distinct ph.id kbsenrollmentid
from
    kbs_billing.purchase_history ph
    join bi_reporting.vw_product_detail prd on ph.product_id = prd.product_id
where
    ph.initial_delta_k_txn_code in ('404', '405', '406');
    --and ph.created_on >= '2018-01-01';
"""

try:
    repeaters
except:
    repeaters = pd.read_sql(sql = query_hsg_repeat, con = engine)
    repeaters['kbsenrollmentid'] = repeaters['kbsenrollmentid'].astype('str')

# Cleaning the data

In [195]:
#If we know the correct answers for all the items in response_df and correct answer column in response_df is not null
#then we can use no_correctAnswer = False
#and by default we work on contentItemId to get multiple correct answers
#we get correct answers df here, we need to filter items that are having multiple versions of items

cor_ans, dup_cor_ans = get_item_cor_ans(response_df, no_correctAnswer = True)

No duplicate correct answer


In [196]:
#CI_old_version_list = dup_cor_ans[['contentItemId']].drop_duplicates(ignore_index=True)

In [197]:
#CI_old_version_dates = pd.DataFrame({'contentItemId':[9132],
#'cutoff_date':['2018-04-18']})
#CI_old_version_dates['cutoff_date'] = CI_old_version_dates['cutoff_date'].astype('datetime64[ns]')
#CI_old_version_dates

In [198]:
#CI_old_keys = pd.DataFrame({'contentItemId':[7420],
#                           'correctAnswer':['3,4']})
CI_old_keys = pd.DataFrame()

## Running cleaning function

In [199]:
data_pool = {'CI_old_version_dates' : pd.DataFrame(),
'CI_old_version_list' : pd.DataFrame(),
'CI_old_keys' : CI_old_keys,
'frt_enrols' : frt_enrols,
'olc_enrols' : olc_enrols,
'repeaters' : repeaters,
'section_map' : pd.DataFrame(),
'test_map' : test_map,
'seqHist_to_exclude' : pd.DataFrame(),
'cidf' : pd.DataFrame(),
'field_test_items' : pd.DataFrame(),
'ci_cols_to_include' : pd.DataFrame(),
'interaction_type_list' : pd.DataFrame()}

#field_test_items should have column name'contentItemName' to match with response_df

In [200]:
result, cleaning_info, rejects_df = clean_item_data(data_path = path,
                         results_path = results_path,
                        analysis_name = analysis_name,
                         resp = response_df,
                         remove_users_deleted_sequences = True,
                         remove_dup_CIs = True,
                         remove_no_kbsEID = True,
                         remove_deleted_sequences = True,
                         remove_impo_response_scored = True,
                         remove_impo_timing_seq = True,
                         remove_seq_w_tmq = True,
                         remove_staged_responses = False,
                         remove_FT_items = False,
                        data_pool = data_pool,
                         CI_remove_before_after = 'before', #applicable for old version dates
                         repeat_treatment = 'omit', #default omit
                         mSec_min_threshold = 5000, #there's y/n condition for timing if none provided
                         mSec_max_threshold = None, #so if condition is not needed.
                         sec_min_threshold = None,
                         sec_max_threshold = None,
                         remove_frt_users = True,
                         remove_olc_users = True,
                         remove_repeat_enrolls = True,
                         remove_tutor = True,
                         remove_ada_seq = True,
                         remove_untimed_seq = True,
                         remove_incomplete_seq = False,
                         seq_item_minutes_threshold = None, #input in terms of minutes here
                         seq_section_minutes_threshold = None, #input in terms of minutes here
                         seq_total_minutes_threshold = None, #input in terms of minutes here
                         qbank = False, #if qbank==false and section_map is not empty then sec_num_attempted is validated against given min_items_per_seq from secton_map
                         min_itmes_per_seq = None, #if qbank==true and section_map provided, sequence_num_attempted is validated against given min_items_per_seq
                         section_calc = True, #if section map provided but min_item_per_seq column not present in it & qbank == false then this gets activated for validating sec_perc_attempted vs section_resp_threshold(comes from section_map)
                         #test_map = test_map, #if section_map is empty this gets activated, and qbank==false then test_resp_threshold comes from test_map
                         #seq_item_resp_threshold = .75,
                         remove_unscored = False,
                         #seqHist_to_exclude = pd.DataFrame(), #should have column sequenceId in it
                         precombined_files = True, #used to add columns from cidf to response_df
                         #cidf = pd.DataFrame(), #pass cidf
                         #interaction_type_list = [], #pass interactionTypeIds in this list
                         #ci_cols_to_include = [], #pass columns names of cidf to include in response_df (ciname, ciid by default added if precombined files is True)
                         remove_repeat_test_administrations = True,
                        remove_seq_wo_dispseq = True)
    

Starting clean item data function at 2021-09-22 16:29:51
Total sequences at start:  1071
Total users at start:  1071
Unique items at start:  100
Total responses at start:  107100

Working on Disqualifiers :
User Removal
--------------------
Sequences with deleted names are removed: NONE

Sequences with dupe content items within the same exam, removed: NONE

Sequences with no KBS EID removed: NONE

Sequence Removal
--------------------
Sequences with deleted names are removed: NONE

Sequences with bad records (response = 0 with score = 1), removed: NONE

Sequences with bad timing (mSecUsed < 0), removed:  11
Users removed  11
Unique items removed  0
Responses removed  1100

Here are the new columns after joining all the test and section maps
{'test_num_ques', 'test_response_threshold', 'test_minutes_allowed'}

Sequences with too many questions in a section, removed: NONE

Item Removal
--------------------
Total item responses:  106000 

Working on Cleaning Rules :
Response / Score Re-co

## Exporting cleaning info & cleaned response_data

In [201]:
#Exporting cleaned response_data
result = result.sort_values(by=['studentId', 'activityId', 'displaySeq'], ignore_index= True)
result.to_csv(results_path+analysis_name+'_responseData.csv', index = False)

#Exporting cleaning info
cleaning_info.to_csv(results_path+analysis_name+'_cleaningInfo.csv')

#Exporting rejects info
rejects_df.sort_values(by=['studentId', 'templateId'], ignore_index = True).to_csv(results_path+analysis_name+'_rejects_info.csv', index = False)

# Creating User Level Matrices

In [202]:
#Reading cleaned data
all_resp = result.copy()

all_resp = all_resp.sort_values(by=['activityName', 'displaySeq'])

#col_name:filename
vars_for_matrix =  {'score':'Item_Scores',
                   'response':'Responses',
                   'mSecUsed':'Milliseconds_per_Item'}

#making matrices
big_matrix = make_user_level_matrices(all_resp,
                               vars_for_matrices = vars_for_matrix,
                               destination_file_path = results_path,
                               destination_file_name_prefix = '_User_level_',
                                analysis_name = analysis_name,
                                omit_code = '.',
                                not_seen_code = '-99',
                                use_display_order = True,
                               qbank = False)

Finished creating matrix for Item_Scores
Finished creating matrix for Responses
Finished creating matrix for Milliseconds_per_Item


# Metadata

## Sequence Level Information

In [203]:
activity_level_info = make_activity_level_info(df = result,
                                              results_path = results_path,
                                              analysis_name = analysis_name)

## User Level Information

In [204]:
user_info = make_user_level_info(df = result,
                                results_path = results_path,
                                analysis_name = analysis_name,
                                test_map = test_map)

## Item Level Information

In [205]:
item_level_info = make_item_level_info(df = result,
                                       qbank = False,
                                      content_df = content_df,
                                      results_path = results_path,
                                      analysis_name = analysis_name,
                                      corr_ans = cor_ans)